# Extract

In [7]:
# Make jupyter to accept import python modules from other folders
import sys
sys.path.append("../../..")

from etl.utils.functions import DB_MinioClient
from etl.utils.constants import Constants
import pandas as pd
import io

FILE_NAME = "cultivos_identificadores.xlsx"
BUCKET_FROM_NAME = Constants.STORAGE_LANDING_ZONE.value

# Connect to MinIO
minio_client = DB_MinioClient().connect()
# Get data from MinIO
data = minio_client.get_object(
    BUCKET_FROM_NAME, FILE_NAME).read()
df = pd.read_excel(io.BytesIO(data), engine="openpyxl",
                   sheet_name="Sheet1", na_values=[''])


ValueError: Worksheet named 'Sheet1' not found

# Clean

In [ ]:
# Change column names
try:
    df.columns = ["agroMeasureId", "plantationKind", "plantationSubKind", "codeType",
        "cropName", "cropVariety", "cropId", "cropVarietyId", "RainfedIrrigated"]
except Exception as e:
    # Finish flow with error
    raise ValueError("Error changing column names: ", e)
# Validate data

# Modify data
# If the column "cropVarietyId" has "***" substitute it with NaN
df["cropVarietyId"] = df["cropVarietyId"].replace(
    to_replace="***", value=None)
# Substitute *** for "Cualquiera"
processed_df = df.replace(to_replace="***", value="Cualquiera")



excel_almendros_cercanos
excel_almendros_cercanos
excel_almendros_cercanos
excel_almendros_cercanos
excel_almendros_cercanos
excel_almendros_cercanos
excel_almendros_cercanos


KeyError: 'x-amz-meta-type'

# Load

In [ ]:
# Connect to MinIO
minio_client = DB_MinioClient().connect()
# Save data to MinIO
with io.BytesIO() as data:
    processed_df.to_excel(data, index=False)
    data.seek(0)
    minio_client.put_object(
        "trusted-zone",
        f"ERP/unknown/{FILE_NAME}",
        data,
        length=data.getbuffer().nbytes,
        content_type="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
        metadata={
            "source": "unknown",
            "type": "trusted"
        }
    )
